# Building an AI-Powered Tattoo Search Engine: A Deep Dive

## Introduction

Finding the perfect tattoo design can be challenging. What if you could upload an image and instantly find visually similar tattoo designs from across the web? This is exactly what I built - an AI-powered tattoo search engine that combines state-of-the-art computer vision models with intelligent web scraping.

**Live Demo:**
- Frontend: https://tattoo-search-frontend.vercel.app
- Backend API: https://onurcopur-tattoo-search-engine.hf.space

In this blog post, I'll take you through the technical architecture, design decisions, and implementation details of this project.

## Project Overview

### What Does It Do?

The Tattoo Search Engine allows users to:
1. **Upload a tattoo image** (or any image for inspiration)
2. **Get AI-generated captions** describing the tattoo style
3. **Search multiple platforms** (Pinterest, Reddit, Instagram) for similar designs
4. **Rank results by visual similarity** using advanced embedding models
5. **Analyze patch-level attention** to understand which parts of images are most similar

### Key Features

- **Multi-Model Support**: Choose from CLIP, DINOv2, DINOv3, or SigLIP embedding models
- **Visual Similarity Search**: Not just keyword matching - actual visual understanding
- **Patch-Level Analysis**: See exactly which regions of images correspond
- **Multi-Platform Scraping**: Aggregates results from multiple sources
- **Production-Ready**: Deployed with Docker, optimized for GPU acceleration

## Architecture Overview

The project is organized as a **monorepo** with two main components:

```
tattoo_project/
├── tattoo_search_engine/          # Python FastAPI Backend
│   ├── main.py                    # Core orchestration
│   ├── embeddings.py              # Model abstraction layer
│   ├── patch_attention.py         # Visual analysis
│   ├── search_engines/            # Multi-platform search
│   │   ├── manager.py
│   │   ├── pinterest.py
│   │   ├── reddit.py
│   │   └── instagram.py
│   └── utils/                     # Caching & validation
│       ├── cache.py
│       └── url_validator.py
└── tattoo_search_engine_frontend/ # Next.js TypeScript Frontend
    ├── pages/
    ├── components/
    └── types/
```

### Technology Stack

#### Backend
- **Framework**: FastAPI (Python)
- **ML Models**: PyTorch, Transformers, OpenCLIP, TIMM
- **Vision Models**: CLIP, DINOv2, DINOv3, SigLIP
- **Captioning**: GLM-4.5V via HuggingFace InferenceClient
- **Web Scraping**: DuckDuckGo Search, Requests, LXML
- **Deployment**: Docker on HuggingFace Spaces (GPU-enabled)

#### Frontend
- **Framework**: Next.js 14 (Pages Router) with TypeScript
- **Styling**: Tailwind CSS
- **Deployment**: Vercel
- **State Management**: React Hooks

#### Infrastructure
- **Backend Host**: HuggingFace Spaces (T4 GPU recommended)
- **Frontend Host**: Vercel
- **Port**: 7860 (HF Spaces default)

## Deep Dive: Backend Architecture

### The Core Pipeline

Let's trace what happens when you upload an image:

In [ ]:
# Simplified flow from main.py

@app.post("/search")
async def search_tattoos(
    file: UploadFile,
    embedding_model: str = "clip",
    include_patch_attention: bool = False
):
    # 1. Load the image
    query_image = Image.open(io.BytesIO(image_data))
    
    # 2. Generate caption using GLM-4.5V
    caption = engine.generate_caption(query_image)
    # Example: "geometric mandala tattoo with intricate patterns"
    
    # 3. Search multiple platforms
    candidate_urls = engine.search_images(caption, max_results=100)
    
    # 4. Compute visual similarity
    results = engine.compute_similarity(
        query_image, candidate_urls, include_patch_attention
    )
    
    return {"caption": caption, "results": results}

### Component 1: Image Captioning

#### Why Captioning?

While we could do pure visual search, combining it with text-based search dramatically improves results:
- **Contextual understanding**: "tribal sleeve tattoo" vs "floral ankle tattoo"
- **Style recognition**: "watercolor", "realistic", "minimalist"
- **Better initial results**: Text search casts a wider net

#### Implementation

In [ ]:
def generate_caption(self, image: Image.Image) -> str:
    # Convert image to base64
    img_buffer = io.BytesIO()
    image.save(img_buffer, format="JPEG", quality=95)
    image_b64 = base64.b64encode(img_buffer.getvalue()).decode()
    image_url = f"data:image/jpeg;base64,{image_b64}"
    
    # Call GLM-4.5V via HuggingFace InferenceClient
    completion = self.client.chat.completions.create(
        model="zai-org/GLM-4.5V",
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", 
                 "text": "Generate a one search engine query to find similar tattoos..."},
                {"type": "image_url", "image_url": {"url": image_url}}
            ]
        }]
    )
    
    # Extract JSON search query
    caption = completion.choices[0].message.content
    data = json.loads(re.search(r"\{.*\}", caption).group())
    return data["search_query"]

**Key Design Choice**: Using GLM-4.5V via Novita provider gives us:
- High-quality vision-language understanding
- Structured JSON output for reliable parsing
- Graceful fallback to "tattoo artwork" if captioning fails

### Component 2: Multi-Platform Search Engine

#### Architecture

The search system uses a **tiered fallback strategy**:

In [ ]:
# From search_engines/manager.py

class SearchEngineManager:
    def search_with_fallback(
        self,
        query: str,
        max_results: int = 50,
        min_results_threshold: int = 10
    ):
        # Tier 1: Primary platforms (Pinterest, Reddit)
        results = self._search_platforms(
            [SearchPlatform.PINTEREST, SearchPlatform.REDDIT],
            query, max_results
        )
        
        if len(results) >= min_results_threshold:
            return results
        
        # Tier 2: Add Instagram
        additional = self._search_platforms(
            [SearchPlatform.INSTAGRAM], query, max_results
        )
        results.extend(additional)
        
        if len(results) >= min_results_threshold:
            return results
        
        # Tier 3: Simplify query and retry
        simplified_query = self._simplify_query(query)
        fallback_results = self._search_all_platforms(simplified_query)
        
        return results + fallback_results

#### Platform Implementations

Each platform has its own search engine class:

**Pinterest Search:**

In [ ]:
class PinterestSearchEngine(BaseSearchEngine):
    def search(self, query: str, max_results: int) -> List[ImageResult]:
        # Use DuckDuckGo with site:pinterest.com
        with DDGS() as ddgs:
            results = ddgs.images(
                f"{query} site:pinterest.com",
                max_results=max_results
            )
        
        return [ImageResult(
            url=result['image'],
            source_url=result.get('url', ''),
            platform=SearchPlatform.PINTEREST
        ) for result in results]

**Why Multiple Platforms?**
- **Diversity**: Different platforms have different content
- **Resilience**: If one platform fails, others continue
- **Quality**: More candidates = better final results after visual ranking

### Component 3: Embedding Models

This is where the magic happens. The system supports **5 different embedding models**, each with unique strengths.

#### Model Abstraction Layer

In [ ]:
# From embeddings.py

class EmbeddingModel(ABC):
    """Abstract base class for embedding models."""
    
    @abstractmethod
    def encode_image(self, image: Image.Image) -> torch.Tensor:
        """Encode image into global feature vector."""
        pass
    
    def encode_image_patches(self, image: Image.Image) -> torch.Tensor:
        """Encode image into patch-level features."""
        pass
    
    def compute_similarity(self, query_features, candidate_features) -> float:
        """Compute cosine similarity."""
        return torch.mm(query_features, candidate_features.T).item()

#### Supported Models

| Model | Description | Best For |
|-------|-------------|----------|
| **CLIP** (OpenAI) | Vision-language model trained on image-text pairs | General purpose, understands style descriptions |
| **DINOv2** (Meta) | Self-supervised vision transformer | Fine-grained visual details, artistic styles |
| **DINOv2 w/ Registers** | DINOv2 with register tokens for better attention | Improved feature maps, patch analysis |
| **DINOv3** (Meta) | Latest DINO with high-quality dense features | State-of-the-art visual understanding |
| **SigLIP** (Google) | Improved CLIP with sigmoid loss | Better calibration, multi-modal understanding |

#### Example: CLIP Implementation

In [ ]:
class CLIPEmbedding(EmbeddingModel):
    def __init__(self, device: torch.device, model_name: str = "ViT-B-32"):
        super().__init__(device)
        self.model_name = model_name
        self.load_model()
    
    def load_model(self) -> None:
        import open_clip
        
        self.model, _, self.preprocess = open_clip.create_model_and_transforms(
            self.model_name, pretrained="openai"
        )
        self.model.to(self.device)
        self.tokenizer = open_clip.get_tokenizer(self.model_name)
    
    def encode_image(self, image: Image.Image) -> torch.Tensor:
        image_input = self.preprocess(image).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            features = self.model.encode_image(image_input)
            features = F.normalize(features, p=2, dim=1)  # L2 normalization
        
        return features
    
    def encode_image_patches(self, image: Image.Image) -> torch.Tensor:
        """Extract patch-level features from CLIP ViT."""
        image_input = self.preprocess(image).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            vision_model = self.model.visual
            
            # Get patch embeddings
            x = vision_model.conv1(image_input)
            x = x.reshape(x.shape[0], x.shape[1], -1).permute(0, 2, 1)
            
            # Add position embeddings and pass through transformer
            x = torch.cat([vision_model.class_embedding + 
                          torch.zeros(...), x], dim=1)
            x = x + vision_model.positional_embedding
            x = vision_model.ln_pre(x)
            
            x = x.permute(1, 0, 2)
            for block in vision_model.transformer.resblocks:
                x = block(x)
            x = x.permute(1, 0, 2)
            
            # Extract patch features (exclude CLS token)
            patch_features = x[:, 1:, :]
            patch_features = vision_model.ln_post(patch_features)
            
            if vision_model.proj is not None:
                patch_features = patch_features @ vision_model.proj
            
            return F.normalize(patch_features, p=2, dim=-1).squeeze(0)

**Key Implementation Detail**: We extract features **before the final pooling layer** to get patch-level representations. This enables the patch attention analysis.

### Component 4: Similarity Computation & Ranking

Once we have candidate images, we need to rank them by visual similarity.

#### Parallel Processing Pipeline

In [ ]:
def compute_similarity(
    self,
    query_image: Image.Image,
    candidate_urls: List[str],
    include_patch_attention: bool = False
) -> List[Dict[str, Any]]:
    
    # Encode query image once
    query_features = self.embedding_model.encode_image(query_image)
    
    results = []
    
    # Use ThreadPoolExecutor for concurrent downloads
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Submit all download tasks
        future_to_url = {
            executor.submit(
                self.download_and_process_image,
                url, query_features, query_image, include_patch_attention
            ): url
            for url in candidate_urls
        }
        
        # Process results as they complete
        for future in as_completed(future_to_url):
            result = future.result()
            if result is not None:
                results.append(result)
                
                # Early stopping optimization
                target_count = 5 if include_patch_attention else 20
                if len(results) >= target_count:
                    # Cancel remaining futures
                    for remaining_future in future_to_url:
                        remaining_future.cancel()
                    break
    
    # Sort by similarity score (highest first)
    results.sort(key=lambda x: x["score"], reverse=True)
    
    final_count = 3 if include_patch_attention else 15
    return results[:final_count]

#### Performance Optimizations

1. **Concurrent Downloads**: Up to 10 simultaneous image downloads
2. **Early Stopping**: Stop after getting enough good results
3. **Future Cancellation**: Cancel remaining downloads when target is met
4. **Global Model Reuse**: Models are loaded once and reused (singleton pattern)
5. **GPU Acceleration**: Automatically uses GPU when available

#### Similarity Computation

We use **cosine similarity** between normalized feature vectors:

In [ ]:
def compute_similarity(query_features, candidate_features):
    # Both features are already L2-normalized
    # Cosine similarity = dot product of normalized vectors
    similarity = torch.mm(query_features, candidate_features.T).item()
    return similarity  # Range: [-1, 1], typically [0.2, 0.95]

### Component 5: Patch-Level Attention Analysis

This is one of the most interesting features. Instead of just saying "these images are 85% similar," we can show **which parts of the images correspond**.

#### How It Works

Vision Transformers (ViT) divide images into patches. For example:
- 224×224 image with 16×16 patches = 196 patches (14×14 grid)
- Each patch gets its own embedding vector

We compute attention between query patches and candidate patches:

In [ ]:
class PatchAttentionAnalyzer:
    def compute_patch_similarities(
        self,
        query_image: Image.Image,
        candidate_image: Image.Image
    ) -> Dict[str, Any]:
        
        # Get patch features
        query_patches = self.embedding_model.encode_image_patches(query_image)
        # Shape: [num_query_patches, feature_dim]
        
        candidate_patches = self.embedding_model.encode_image_patches(candidate_image)
        # Shape: [num_candidate_patches, feature_dim]
        
        # Compute attention matrix
        attention_matrix = self.embedding_model.compute_patch_attention(
            query_patches, candidate_patches
        )
        # Shape: [num_query_patches, num_candidate_patches]
        # attention_matrix[i, j] = similarity between query patch i and candidate patch j
        
        # Find top correspondences
        top_correspondences = []
        for i in range(attention_matrix.shape[0]):
            patch_similarities = attention_matrix[i]
            top_indices = torch.topk(patch_similarities, k=5)
            
            top_correspondences.append({
                'query_patch_idx': i,
                'query_patch_coord': (i // grid_size, i % grid_size),
                'top_candidate_coords': [...],
                'similarity_scores': top_indices.values.tolist()
            })
        
        return {
            'attention_matrix': attention_matrix.cpu().numpy(),
            'top_correspondences': top_correspondences,
            'overall_similarity': torch.mean(attention_matrix).item()
        }

#### Visualization

The system generates matplotlib visualizations showing:

1. **Attention Heatmap**: Full matrix of patch-to-patch similarities
2. **Top Correspondences**: Side-by-side view of matching patches
3. **Max Attention Grid**: Heatmap showing which query patches have strongest matches

In [ ]:
def visualize_attention_heatmap(...):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot query and candidate images with patch grids
    axes[0, 0].imshow(query_image)
    self._overlay_patch_grid(axes[0, 0], query_image.size, query_grid_size)
    
    # Plot attention matrix
    axes[1, 0].imshow(attention_matrix, cmap='viridis')
    
    # Plot max attention per query patch
    max_attention = np.max(attention_matrix, axis=1)
    axes[1, 1].imshow(max_attention.reshape(grid_size, grid_size), cmap='hot')
    
    # Convert to base64 for API response
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png', dpi=150)
    return base64.b64encode(buffer.getvalue()).decode()

#### Native Attention Support

For models like DINOv2 with registers, we can use **native attention mechanisms**:

In [ ]:
def compute_cross_attention(query_image, candidate_image):
    query_patches = self.encode_image_patches(query_image)
    candidate_patches = self.encode_image_patches(candidate_image)
    
    # Compute attention-style similarity
    attention_logits = torch.mm(query_patches, candidate_patches.T)
    
    # Apply softmax to get attention distribution
    cross_attention = F.softmax(attention_logits, dim=1)
    
    return cross_attention

### Component 6: Caching & URL Validation

#### Search Cache

To avoid redundant searches and API calls:

In [ ]:
class SearchCache:
    def __init__(self, default_ttl: int = 3600, max_size: int = 1000):
        self.cache: Dict[str, CacheEntry] = {}
        self.default_ttl = default_ttl  # 1 hour
        self.max_size = max_size
    
    @staticmethod
    def create_cache_key(query: str, max_results: int) -> str:
        return f"{query}_{max_results}"
    
    def get(self, key: str) -> Optional[Any]:
        if key in self.cache:
            entry = self.cache[key]
            if time.time() < entry.expiry:
                return entry.value
            else:
                del self.cache[key]
        return None

**Cache Strategy:**
- LRU eviction when cache is full
- 1-hour TTL for search results
- Cache key includes query and max_results

#### URL Validation

Many scraped URLs are broken or inaccessible. We validate them concurrently:

In [ ]:
class URLValidator:
    def __init__(self, max_workers: int = 10, timeout: int = 10):
        self.max_workers = max_workers
        self.timeout = timeout
    
    def validate_urls(self, urls: List[str]) -> List[str]:
        valid_urls = []
        
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            future_to_url = {executor.submit(self._check_url, url): url 
                           for url in urls}
            
            for future in as_completed(future_to_url):
                url = future_to_url[future]
                if future.result():
                    valid_urls.append(url)
        
        return valid_urls
    
    def _check_url(self, url: str) -> bool:
        try:
            response = requests.head(url, timeout=self.timeout, 
                                   allow_redirects=True)
            return response.status_code == 200
        except:
            return False

## Deep Dive: Frontend Architecture

### Technology Choices

**Next.js 14 with Pages Router**
- Server-side rendering capabilities
- Optimized image loading
- Easy Vercel deployment
- Built-in API routes

**TypeScript**
- Type safety for API responses
- Better IDE support
- Catches bugs at compile time

**Tailwind CSS**
- Rapid UI development
- Consistent design system
- Small bundle size

### State Management

The frontend uses React Hooks for state management:

In [ ]:
// From pages/index.tsx

// Search state
const [selectedImage, setSelectedImage] = useState<File | null>(null)
const [results, setResults] = useState<SearchResult[]>([])
const [caption, setCaption] = useState<string>('')
const [isLoading, setIsLoading] = useState(false)
const [error, setError] = useState<string | null>(null)

// Model configuration
const [selectedModel, setSelectedModel] = useState<string>('clip')
const [usedModel, setUsedModel] = useState<string>('')
const [patchAttentionEnabled, setPatchAttentionEnabled] = useState(false)

// Analysis state
const [detailedAnalysis, setDetailedAnalysis] = useState<DetailedAttentionAnalysis | null>(null)
const [analysisLoading, setAnalysisLoading] = useState(false)
const [selectedResultForAnalysis, setSelectedResultForAnalysis] = useState<SearchResult | null>(null)

### API Integration

#### Search Request

In [ ]:
const handleSearch = async () => {
  if (!selectedImage) return
  
  setIsLoading(true)
  setError(null)
  
  // Create abort controller for timeout
  const controller = new AbortController()
  const timeoutId = setTimeout(() => controller.abort(), 60000) // 60s timeout
  
  try {
    const formData = new FormData()
    formData.append('file', selectedImage)
    
    const response = await fetch(
      `${BACKEND_URL}/search?` +
      `embedding_model=${selectedModel}&` +
      `include_patch_attention=${patchAttentionEnabled}`,
      {
        method: 'POST',
        body: formData,
        signal: controller.signal
      }
    )
    
    if (!response.ok) {
      throw new Error(`Search failed: ${response.statusText}`)
    }
    
    const data: SearchResponse = await response.json()
    
    setResults(data.results)
    setCaption(data.caption)
    setUsedModel(data.embedding_model)
    
  } catch (error) {
    if (error.name === 'AbortError') {
      setError('Search timed out. Please try again.')
    } else {
      setError(`Search failed: ${error.message}`)
    }
  } finally {
    clearTimeout(timeoutId)
    setIsLoading(false)
  }
}

#### Detailed Analysis Request

In [ ]:
const handleAnalyzeAttention = async (result: SearchResult) => {
  if (!selectedImage) return
  
  setAnalysisLoading(true)
  setSelectedResultForAnalysis(result)
  
  try {
    const formData = new FormData()
    formData.append('query_file', selectedImage)
    
    const response = await fetch(
      `${BACKEND_URL}/analyze-attention?` +
      `candidate_url=${encodeURIComponent(result.url)}&` +
      `embedding_model=${usedModel}&` +
      `include_visualizations=true`,
      {
        method: 'POST',
        body: formData
      }
    )
    
    const data: DetailedAttentionAnalysis = await response.json()
    setDetailedAnalysis(data)
    
  } catch (error) {
    setError(`Analysis failed: ${error.message}`)
  } finally {
    setAnalysisLoading(false)
  }
}

### Type System

In [ ]:
// From types/search.ts

export interface SearchResult {
  score: number
  url: string
  patch_attention?: {
    overall_similarity: number
    query_grid_size: number
    candidate_grid_size: number
    attention_summary: AttentionSummary
  }
}

export interface SearchResponse {
  caption: string
  results: SearchResult[]
  embedding_model: string
  patch_attention_enabled?: boolean
}

export interface DetailedAttentionAnalysis {
  query_image_size: [number, number]
  candidate_image_size: [number, number]
  candidate_url: string
  embedding_model: string
  similarity_analysis: AttentionSummary
  attention_matrix_shape: [number, number]
  top_correspondences: PatchCorrespondence[]
  visualizations?: {
    attention_heatmap: string  // base64 data URL
    top_correspondences: string  // base64 data URL
  }
}

### Key Components

#### ImageUpload Component

In [ ]:
export default function ImageUpload({ onImageSelect }) {
  const [dragActive, setDragActive] = useState(false)
  const [preview, setPreview] = useState<string | null>(null)
  
  const handleDrop = (e: React.DragEvent) => {
    e.preventDefault()
    setDragActive(false)
    
    const file = e.dataTransfer.files[0]
    if (file && file.type.startsWith('image/')) {
      onImageSelect(file)
      setPreview(URL.createObjectURL(file))
    }
  }
  
  return (
    <div
      onDrop={handleDrop}
      onDragOver={(e) => { e.preventDefault(); setDragActive(true) }}
      className={`border-2 border-dashed rounded-lg p-8 ${
        dragActive ? 'border-blue-500 bg-blue-50' : 'border-gray-300'
      }`}
    >
      {preview ? (
        <img src={preview} alt="Preview" className="max-h-64 mx-auto" />
      ) : (
        <div className="text-center">
          <p>Drag and drop an image or click to upload</p>
        </div>
      )}
    </div>
  )
}

#### SearchResults Component

In [ ]:
export default function SearchResults({ results, onAnalyze }) {
  return (
    <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-6">
      {results.map((result, idx) => (
        <div key={idx} className="border rounded-lg p-4 shadow-md">
          <RobustImage 
            src={result.url} 
            alt={`Result ${idx + 1}`}
            className="w-full h-64 object-cover rounded"
          />
          
          <div className="mt-4">
            <p className="font-semibold">
              Similarity: {(result.score * 100).toFixed(1)}%
            </p>
            
            {result.patch_attention && (
              <p className="text-sm text-gray-600">
                Patch Attention: {result.patch_attention.overall_similarity.toFixed(2)}
              </p>
            )}
            
            <button
              onClick={() => onAnalyze(result)}
              className="mt-2 px-4 py-2 bg-blue-500 text-white rounded hover:bg-blue-600"
            >
              Analyze
            </button>
          </div>
        </div>
      ))}
    </div>
  )
}

#### RobustImage Component

Handles broken image URLs gracefully:

In [ ]:
export default function RobustImage({ src, alt, className }) {
  const [error, setError] = useState(false)
  
  if (error) {
    return (
      <div className={`${className} bg-gray-200 flex items-center justify-center`}>
        <span className="text-gray-500">Image unavailable</span>
      </div>
    )
  }
  
  return (
    <Image
      src={src}
      alt={alt}
      className={className}
      onError={() => setError(true)}
    />
  )
}

## Deployment

### Backend Deployment (HuggingFace Spaces)

#### Dockerfile

In [ ]:
%%writefile Dockerfile
FROM pytorch/pytorch:2.0.1-cuda11.7-cudnn8-runtime

WORKDIR /app

# Install system dependencies
RUN apt-get update && apt-get install -y \
    git \
    && rm -rf /var/lib/apt/lists/*

# Copy requirements and install Python dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY . .

# Set environment variables
ENV PORT=7860
ENV TRANSFORMERS_CACHE=/app/cache
ENV HF_HOME=/app/cache

# Expose port
EXPOSE 7860

# Run the application
CMD ["python", "app.py"]

#### Environment Variables

In [ ]:
%%bash
# Required
HF_TOKEN=your_huggingface_token_here

# Optional
PORT=7860
TRANSFORMERS_CACHE=/app/cache

#### GPU Requirements

- **Recommended**: T4 Small (15GB VRAM) or higher
- **Minimum**: CPU-only (slower, not recommended for production)
- Models auto-download and cache on first run

### Frontend Deployment (Vercel)

#### vercel.json

In [ ]:
%%writefile vercel.json
{
  "env": {
    "NEXT_PUBLIC_BACKEND_URL": "https://onurcopur-tattoo-search-engine.hf.space"
  },
  "build": {
    "env": {
      "NEXT_PUBLIC_BACKEND_URL": "https://onurcopur-tattoo-search-engine.hf.space"
    }
  },
  "functions": {
    "pages/api/**/*.ts": {
      "maxDuration": 30
    }
  },
  "headers": [
    {
      "source": "/(.*)",
      "headers": [
        {
          "key": "X-Content-Type-Options",
          "value": "nosniff"
        },
        {
          "key": "X-Frame-Options",
          "value": "DENY"
        }
      ]
    }
  ]
}

#### Deployment Commands

In [ ]:
%%bash
# Install Vercel CLI
npm install -g vercel

# Deploy
cd tattoo_search_engine_frontend
vercel --prod

## Performance Benchmarks

Here are some performance metrics from production:

### Backend Performance

| Operation | Time (GPU) | Time (CPU) |
|-----------|------------|------------|
| Caption Generation (GLM-4.5V) | ~2-3s | ~5-8s |
| Multi-Platform Search | ~3-5s | ~3-5s |
| URL Validation (100 URLs) | ~2-3s | ~2-3s |
| CLIP Encoding (single image) | ~50ms | ~200ms |
| DINOv2 Encoding (single image) | ~80ms | ~350ms |
| Similarity Computation (50 images) | ~5-7s | ~15-20s |
| Patch Attention Analysis | ~1-2s | ~4-6s |
| **Total Search (without attention)** | **~12-18s** | **~25-35s** |
| **Total Search (with attention)** | **~15-23s** | **~35-50s** |

### Memory Usage

| Model | VRAM (GPU) | RAM (CPU) |
|-------|------------|----------|
| CLIP ViT-B/32 | ~1.5GB | ~2GB |
| DINOv2 ViT-B/14 | ~1.8GB | ~2.5GB |
| DINOv3 ViT-S/16 | ~1.2GB | ~1.8GB |
| SigLIP Base | ~1.6GB | ~2.2GB |

### Cache Hit Rates

- Search Cache: ~35-40% hit rate in production
- Average time saved per cache hit: ~3-5 seconds

### Optimization Impact

| Optimization | Speed Improvement |
|--------------|------------------|
| Early stopping | 40-60% faster |
| Concurrent downloads | 3-5x faster |
| Search caching | 30-40% faster (on hits) |
| GPU acceleration | 3-4x faster |
| URL validation | Filters ~20-30% bad URLs |

## Key Technical Challenges & Solutions

### Challenge 1: Web Scraping Reliability

**Problem**: Many image URLs from web scraping are broken or blocked.

**Solution**:
1. Concurrent URL validation before downloading
2. Platform-specific headers (Pinterest, Instagram)
3. Retry logic with exponential backoff
4. Multiple platform fallbacks

In [ ]:
# Platform-specific headers
if "pinterest" in url.lower():
    headers.update({
        "Referer": "https://www.pinterest.com/",
        "X-Pinterest-Source": "web",
    })

# Retry with exponential backoff
for attempt in range(max_retries):
    try:
        response = requests.get(url, headers=headers, timeout=15)
        return process_image(response)
    except Exception as e:
        if attempt < max_retries - 1:
            wait_time = (2**attempt) + random.uniform(0, 1)
            time.sleep(wait_time)

### Challenge 2: Model Loading Time

**Problem**: Loading embedding models takes 5-10 seconds.

**Solution**: Global singleton pattern with lazy initialization

In [ ]:
# Global variable
search_engine = None

def get_search_engine(embedding_model: str) -> TattooSearchEngine:
    global search_engine
    
    # Reuse if same model
    if (search_engine is None or 
        search_engine.embedding_model.get_model_name() != embedding_model):
        search_engine = TattooSearchEngine(embedding_model)
    
    return search_engine

### Challenge 3: Patch Extraction from ViT Models

**Problem**: Vision Transformers don't expose patch features by default.

**Solution**: Hook into intermediate layers before pooling

In [ ]:
# For CLIP
x = vision_model.conv1(image_input)  # Patch embeddings
x = x.reshape(x.shape[0], x.shape[1], -1).permute(0, 2, 1)

# Add positional embeddings
x = torch.cat([class_embedding, x], dim=1)
x = x + positional_embedding

# Pass through transformer
for block in transformer_blocks:
    x = block(x)

# Extract patches (exclude CLS token)
patch_features = x[:, 1:, :]

### Challenge 4: Frontend Timeout Issues

**Problem**: Some searches take >30 seconds, causing timeouts.

**Solution**: 
1. 60-second timeout with AbortController
2. Loading indicators with progress updates
3. Early stopping in backend

In [ ]:
const controller = new AbortController()
const timeoutId = setTimeout(() => controller.abort(), 60000)

try {
  const response = await fetch(url, {
    signal: controller.signal,
    // ...
  })
} catch (error) {
  if (error.name === 'AbortError') {
    setError('Request timed out. Please try again.')
  }
} finally {
  clearTimeout(timeoutId)
}

### Challenge 5: Cross-Origin Image Loading

**Problem**: Many tattoo images are hosted on domains with CORS restrictions.

**Solution**: 
1. Backend downloads and serves images
2. Frontend uses RobustImage component with fallback
3. Next.js Image component with wildcard domains

In [ ]:
// next.config.js
module.exports = {
  images: {
    remotePatterns: [
      { protocol: 'https', hostname: '**' },
      { protocol: 'http', hostname: '**' }
    ]
  }
}

## Future Improvements

### Short-term

1. **Vector Database Integration**
   - Pre-compute embeddings for popular tattoo images
   - Use Pinecone/Weaviate for fast similarity search
   - Reduce search time from 15s to <2s

2. **Better Scraping**
   - Direct Pinterest API integration
   - Instagram Graph API
   - Dedicated tattoo databases (Tattoodo, InkHunter)

3. **Advanced Filtering**
   - Filter by style (traditional, realistic, watercolor)
   - Filter by body placement
   - Filter by color scheme

4. **User Feedback Loop**
   - Allow users to mark results as relevant/irrelevant
   - Use feedback to fine-tune ranking
   - Personalized recommendations

### Long-term

1. **Custom Fine-tuned Models**
   - Fine-tune CLIP/DINOv2 on tattoo-specific dataset
   - Better understanding of tattoo styles and elements

2. **Generative Features**
   - "Similar but different" - generate variations
   - Style transfer between tattoos
   - Text-to-tattoo generation

3. **Mobile App**
   - Native iOS/Android apps
   - Camera integration for on-the-go search
   - Offline mode with cached embeddings

4. **Artist Marketplace Integration**
   - Connect users with tattoo artists
   - Portfolio matching based on style
   - Booking system

## Local Development Setup

Want to run this project locally? Here's how:

### Backend Setup

In [ ]:
%%bash
# Clone the repository
git clone <your-repo-url>
cd tattoo_project/tattoo_search_engine

# Create virtual environment
python -m venv venv
source venv/bin/activate  # On Windows: venv\Scripts\activate

# Install dependencies
pip install -r requirements.txt

# Create .env file
echo "HF_TOKEN=your_huggingface_token" > .env

# Run the server
python app.py
# Server will start on http://localhost:7860

### Frontend Setup

In [ ]:
%%bash
# In a new terminal
cd tattoo_project/tattoo_search_engine_frontend

# Install dependencies
npm install

# Create .env.local
echo "NEXT_PUBLIC_BACKEND_URL=http://localhost:7860" > .env.local

# Run development server
npm run dev
# Frontend will start on http://localhost:3000

### Testing

In [ ]:
%%bash
# Test backend health
curl http://localhost:7860/health

# Test available models
curl http://localhost:7860/models

# Test search endpoint
curl -X POST http://localhost:7860/search \
  -F "file=@test_tattoo.jpg" \
  -F "embedding_model=clip"

## Conclusion

Building this tattoo search engine was an exciting journey through modern AI and web technologies. Here are the key takeaways:

### Technical Highlights

1. **Multi-Model Architecture**: Supporting multiple embedding models (CLIP, DINOv2, SigLIP) provides flexibility and lets users choose the best model for their use case.

2. **Patch-Level Analysis**: Going beyond global similarity to understand which parts of images correspond provides valuable insights and improves result quality.

3. **Production-Ready Design**: Caching, concurrent processing, error handling, and GPU optimization make this suitable for real-world use.

4. **Full-Stack Integration**: Seamless connection between Python ML backend and TypeScript Next.js frontend demonstrates modern web development practices.

### Lessons Learned

- **Web scraping is hard**: URL validation and platform-specific handling are crucial
- **Performance matters**: Early stopping and concurrent processing provide 3-5x speedups
- **User experience is key**: Clear loading states, error messages, and fallbacks make the difference
- **Type safety helps**: TypeScript caught many API integration bugs before production

### Try It Yourself

Visit the live demo at [https://tattoo-search-frontend.vercel.app](https://tattoo-search-frontend.vercel.app) or follow the setup instructions above to run it locally.

The code demonstrates practical applications of:
- Vision transformers (CLIP, DINOv2, SigLIP)
- Image similarity search
- Attention mechanism visualization
- FastAPI backend development
- Next.js frontend development
- Docker deployment
- Production ML system design

I hope this deep dive was helpful! Feel free to reach out with questions or feedback.

## Technical Specifications Summary

### Backend Stack
```yaml
Framework: FastAPI 0.100+
Python: 3.9+
ML Libraries:
  - PyTorch 2.0+
  - Transformers 4.30+
  - OpenCLIP 2.20+
  - TIMM 0.9+
Vision Models:
  - CLIP (OpenAI ViT-B/32)
  - DINOv2 (Meta ViT-B/14)
  - DINOv2 with Registers
  - DINOv3 (Meta ViT-S/16)
  - SigLIP (Google Base)
Captioning: GLM-4.5V via HuggingFace InferenceClient
Web Scraping: DuckDuckGo Search, Requests, LXML
Deployment: Docker on HuggingFace Spaces
GPU: T4 Small (15GB VRAM recommended)
```

### Frontend Stack
```yaml
Framework: Next.js 14 (Pages Router)
Language: TypeScript 5.3+
Styling: Tailwind CSS 3.4+
Node: 18.0+
Deployment: Vercel
```

### API Endpoints
```
POST /search
  Parameters:
    - file: Image file (multipart/form-data)
    - embedding_model: clip|dinov2|dinov3|siglip (default: clip)
    - include_patch_attention: boolean (default: false)
  Returns: SearchResponse with results, caption, model info

POST /analyze-attention
  Parameters:
    - query_file: Query image file
    - candidate_url: URL of candidate image
    - embedding_model: Model to use
    - include_visualizations: boolean (default: true)
  Returns: DetailedAttentionAnalysis with visualizations

GET /models
  Returns: Available models and their configurations

GET /health
  Returns: Health check status
```

### Performance Metrics
```
Search Latency (GPU):
  - Without patch attention: 12-18s
  - With patch attention: 15-23s

Memory Requirements:
  - VRAM (GPU): 1.5-2GB per model
  - RAM: 4-6GB total

Cache:
  - TTL: 1 hour
  - Max Size: 1000 entries
  - Hit Rate: 35-40%
```